### File Description

#### Author: Ajay Ragh
#### Packages: ionex,numpy,matplotlib
##### ionex: https://github.com/gnss-lab/ionex.git
##### Comments: uncomment next cell to install the ionex package
##### Date: 07-04-2022

In [1]:
# !pip install -e git+https://github.com/gnss-lab/ionex.git#egg=ionex

In [2]:
import ionex as ix
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [3]:
def tec_grid(file):
    inx = ix.reader(file)
    exponent = inx.exponent
    tec  = []
    epochs = []
    for i in inx:
        vals = i.tec
        epoch_val = i.epoch
        tec.append(vals)
        epochs.append(epoch_val)
    height = i.height
    print("the altitude of data measured is:",height)
    tec = np.asarray(tec)
    
    return inx,tec,epochs,exponent

In [4]:
def get_lats_lons(inx):
    lat = inx.latitude
    lon = inx.longitude
    lats = []
    lats.append(lat.lat1)
    count_lat = lat.lat1
    for i in range (int((abs(lat.lat1-lat.lat2)/abs(lat.dlat)))):
        count_lat = count_lat+lat.dlat
        lats.append(count_lat)
    
    lons = []
    lons.append(lon.lon1)
    count_lon = lon.lon1
    for i in range (int((abs(lon.lon1-lon.lon2)/abs(lon.dlon)))):
        count_lon = count_lon+lon.dlon
        lons.append(count_lon)
    
    return lats,lons

In [5]:
def get_tec(file,epoch,lat,lon):
    inx,tec,epochs,exponent = tec_grid(file)
    lats,lons = get_lats_lons(inx)
    tec = tec.reshape(tec.shape[0],len(lats),len(lons))
    closest_lat = lats[min(range(len(lats)), key = lambda i: abs(lats[i]-lat))]
    closest_lon = lons[min(range(len(lons)), key = lambda i: abs(lons[i]-lon))]
    
    print("Closest latitude:",closest_lat)
    print("Closest Longitude:",closest_lon)
    
    lat_index=lats.index(closest_lat)
    lon_index=lons.index(closest_lon)
    epoch_index = epochs.index(epoch)
    
    print("Epoch and index:",epoch,",",epoch_index)
    
    tec_val = tec[epoch_index,lat_index,lon_index]
    tec_val = tec_val*10**(exponent)
    
    return tec_val

In [6]:
def show_grid(file):
    temp_val = ix.reader(file)
    for i in temp_val:
        print(i.grid)
        break

In [7]:
file = "data/igsg0210.22i"
show_grid(file)


Grid(latitude=Latitude(lat1=87.5, lat2=-87.5, dlat=-2.5), longitude=Longitude(lon1=-180.0, lon2=180.0, dlon=5.0))


### Enter latitude between lat1 & lat2, similarly enter longitude between lon1 & lon2. 
### The displayed tec value will be for the closest latitude and longitude in the data to the entered latitude and longitude


In [8]:
#start epoch = 2022/1/21,00:00
#epoch increment = 2 hours
#enter epoch accordingly
yyyy=2022
mm=1
dd=21
hh=14
mins=0

lat= 12
lon=-163
epoch=datetime.datetime(yyyy,mm,dd,hh,mins)

tec_val = get_tec(file,epoch,lat,lon)
print("TEC Value:",tec_val)

the altitude of data measured is: Height(hgt1=450.0, hgt2=450.0, dhgt=0.0)
Closest latitude: 12.5
Closest Longitude: -165.0
Epoch and index: 2022-01-21 14:00:00 , 7
TEC Value: 0.7400000000000001


#### Visualising TEC layers

In [9]:
plt.rcParams["figure.figsize"] = (20,20)

inx,tec,epochs= tec_grid(file)
lats,lons = get_lats_lons(inx)
tec = tec.reshape(tec.shape[0],len(lats),len(lons))
for i in range(tec.shape[0]):
    plt.subplot(3,5,i+1)
    plt.imshow(tec[i])

the altitude of data measured is: Height(hgt1=450.0, hgt2=450.0, dhgt=0.0)


ValueError: too many values to unpack (expected 3)